# Extract ToposText Annotations in Book 4

In order to create a new dataset of place names and evaluate the quality of the ToposText annotation, we extracted all the annotations in Book 4 including places, persons, and ethnics. For each annotation, we extracted its position (book, chapter, paragraph), the textual content of the tag (ie., Rome), the class label (if present, ie. 'place'), the start and end position of the word in the paragraph, and the corresponding ToposText ID (if present).

The position of the word in the paragraph is calculated starting from the very beginning of the paragraph. Please notice that in ToposText each paragraph begins with the ID of the paragraph (for instance, § 4.1.1 EPIRUS: ...). Punctuation was not removed.

To extract the start position of the tagged word, we started from the HTML version of the text containing the tags. Then, each paragraph was cleaned removing the p tag (tag of the paragraph), the b tag (tag of the paragraph ID), and the content of the a tags (tags of places, people, ethnics). We substituted the < a > tag with a special character (+) to mark the following word as a 'tagged word'. The position of the special character (+) is the start position of the tagged word minus the number of all the special characters preceding the tagged word present in the paragraph. At the end of the process, we obtained a list of all the start positions of tagged words.

Splitting the processes of (1) start position extraction and (2) tagged word extraction was necessary to overcome some difficulties we faced in extracting the start position of words in a tagged text in which some of the tagged words occur more than once.

The ToposText IDs are annotated in two different ways. In some tags, the ToposText ID is in the "about" element. In other cases, it is in the "hfref" element.

In total, 1,888 annotated words were extracted from Book 4.

In [1]:
import pandas as pd
import re
import csv
from bs4 import BeautifulSoup

# 1.1.1 Extraction of Start Positions of all the Tagged Words

The next cell contains the code to extract the start and end position of a word between tags in an HTML source page. 

In [2]:
## open the source HTML page as soup by BeautifulSoup
soup = BeautifulSoup(open("/Users/u0154817/OneDrive - KU Leuven/Documents/KU Leuven/PhD project 'Greek Spaces in Roman Times'/Data_Extraction/Sources/NH_Eng_ToposText/NH_Eng_1-11.html", encoding='utf-8'), features="lxml")

## get all the paragraphs in Book 4
Book_4 = soup.find_all("p", id=lambda x: x and x.startswith("urn:cts:latinLit:phi0978.phi001:4.")) ## get all the paragraph starting with the ID phi0978.phi001:4.

## create a list of the start positions of all the tagged words in Book 4
Start_Positions_Annotations_Book4 = []

for Paragraph in Book_4: ## for each paragraph in Book 4
    
    Paragraph=str(Paragraph) ## convert the paragraph to a string
    
    ## clean the text
    Clean_text = re.sub('<p[^>]*>', '', Paragraph) ## remove the p tag
    Clean_text = re.sub('</p>', '', Clean_text)

    Clean_text = re.sub('<b>', '', Clean_text) ## remove the b tag
    Clean_text = re.sub('</b>', '', Clean_text)
    
    Clean_text = re.sub('</a>', '', Clean_text) ## remove the </a> tag
    Clean_text = re.sub('<a[^>]*>', '+', Clean_text) ## substitute the head of the <a> tag with the special character +
    
    print(Clean_text) ## show the cleansed text
    
    List_of_Start_Positions = [] ## list of the start positions of tagged words in the paragraph
    List_of_Special_Char_Seen = 0 ## list of the number of special characters (+) already seen
    
    for i, Char in enumerate(Clean_text): ## for each character in the paragraph
        if Char == "+" : ## if the character is the special character +
            List_of_Start_Positions.append(i - List_of_Special_Char_Seen) ## the start position of the following tagged word is the position of the special character minus the number of all the special characters alreadty seen
            List_of_Special_Char_Seen += 1 ## add +1 to the sum of the special characters already seen
            
    print(List_of_Start_Positions)
    Start_Positions_Annotations_Book4.extend(List_of_Start_Positions) ## append the list of start position

§ 4.1.1  EPIRUS: The third great Gulf of Europe begins at the mountains of +Acroceraunia, and ends at the +Hellespont, embracing an extent of 2500 miles, exclusive of the sea-line of nineteen smaller gulfs. Upon it are +Epirus, +Acarnania, +Aitolia, +Phocis, +Locris, +Achaia, +Messenia, +Laconia, +Argolis, +Megaris, +Attica, +Boeotia; and again, upon the other sea, the same +Phocis and +Locris, +Doris, +Phthiotis, +Thessalia, +Magnesia, +Macedonia and Thracia. All the fabulous lore of Greece, as well as the effulgence of her literature, first shone forth upon the banks of this Gulf. We shall therefore dwell a little the longer upon it. 
[75, 105, 217, 225, 236, 245, 253, 261, 269, 279, 288, 297, 306, 314, 363, 374, 382, 389, 400, 411, 421]
§ 4.1.2  +Epirus, generally so called, begins at the mountains of +Acroceraunia. The first people that we meet are the +Chaones, from whom +Chaonia receives its name, then the +Thesproti, and then the Antigonenses. We then come to the place where Aor

In total, 1,888 start positions of tagged words were extracted from Book 4.

In [ ]:
len(Start_Positions_Annotations_Book4)

# 1.1.2 Extract all the Annotations

The next cell contains the code to extract tagged words from a ToposText source page and create a CSV file with the extracted information.

In [3]:
## open the source HTML page as soup by BeautifulSoup
soup = BeautifulSoup(open("/Users/u0154817/OneDrive - KU Leuven/Documents/KU Leuven/PhD project 'Greek Spaces in Roman Times'/Data_Extraction/Sources/NH_Eng_ToposText/NH_Eng_1-11.html", encoding='utf-8'), features="lxml")

## write the new csv file
f = csv.writer(open("1.1.ToposText_Annotations_Book_4_TEMP.csv", "w", newline=''))
## define column headers in the csv file
f.writerow(["Reference", "Tagged Entity", "Class", "Start position", "End position", "ToposText ID", "Temporary_ToposTextID_href"])

## get all the paragraphs in Book 4
Book_4 = soup.find_all("p", id=lambda x: x and x.startswith("urn:cts:latinLit:phi0978.phi001:4."))

Count_Annotations = 0 ## count the number of annotations detected

for Paragraph in Book_4: ## for each paragraph in Book 4 

    Reference = Paragraph.get("id") ## get the ID of the paragraph (book, chapter, paragraph)
    a_tags = Paragraph.find_all('a') ## get all a tags in the paragraph
    
    for a_tag in a_tags: ## for each a tag
        
        Tagged_Entity = a_tag.get_text() ## get the word content in the tag
        Class = a_tag.get('class') ## get the class
        Start_Position = Start_Positions_Annotations_Book4[Count_Annotations] ## get the start position in the list created above, the index is equal to the number of annotations already detected
        End_Position = Start_Position+len(Tagged_Entity) ## the end position is equal to the sum of start pos and the lenght of the string
        
        ToposText_ID_about = a_tag.get('about') ## extract the content of "about"
        ToposText_ID_href = a_tag.get('href') ## extract the content of "href"
        
        if ToposText_ID_href : ## if the tag contains "href"
            ToposText_ID_href = "https://topostext.org"+ToposText_ID_href ## create the complete link of href
        
        f.writerow([Reference, Tagged_Entity, Class, Start_Position, End_Position, ToposText_ID_about, ToposText_ID_href])
        Count_Annotations += 1
        
        print(Reference, Tagged_Entity, Class, Start_Position, End_Position, ToposText_ID_about, ToposText_ID_href)

urn:cts:latinLit:phi0978.phi001:4.1.1 Acroceraunia ['place'] 75 87 https://topostext.org/place/404194LKer None
urn:cts:latinLit:phi0978.phi001:4.1.1 Hellespont ['place'] 105 115 https://topostext.org/place/402264WHel None
urn:cts:latinLit:phi0978.phi001:4.1.1 Epirus ['place'] 217 223 https://topostext.org/place/395205REpe None
urn:cts:latinLit:phi0978.phi001:4.1.1 Acarnania ['place'] 225 234 https://topostext.org/place/388210RAka None
urn:cts:latinLit:phi0978.phi001:4.1.1 Aitolia ['place'] 236 243 https://topostext.org/place/386215RAit None
urn:cts:latinLit:phi0978.phi001:4.1.1 Phocis ['place'] 245 251 https://topostext.org/place/386227RPho None
urn:cts:latinLit:phi0978.phi001:4.1.1 Locris ['place'] 253 259 https://topostext.org/place/384222ROzo None
urn:cts:latinLit:phi0978.phi001:4.1.1 Achaia ['place'] 261 267 https://topostext.org/place/382219RAch None
urn:cts:latinLit:phi0978.phi001:4.1.1 Messenia ['place'] 269 277 https://topostext.org/place/372218RMes None
urn:cts:latinLit:phi097

In [4]:
Count_Annotations

1888

In [5]:
## read the csv file
ToposText_Book4 = pd.read_csv("/Users/u0154817/OneDrive - KU Leuven/Documents/KU Leuven/PhD project 'Greek Spaces in Roman Times'/Data_Extraction/Python Scripts/1.1.ToposText_Annotations_Book_4_TEMP.csv", delimiter=",")
len(ToposText_Book4)

1888

In [6]:
## replace missing values in ToposText ID with values from Temporary_ToposTextID_href
ToposText_Book4['ToposText ID'] = ToposText_Book4['ToposText ID'].fillna(ToposText_Book4['Temporary_ToposTextID_href'])

In [7]:
len(ToposText_Book4)

1888

In [9]:
ToposText_Book4 = ToposText_Book4.drop(['Temporary_ToposTextID_href'], axis=1)

In [10]:
ToposText_Book4.to_csv("1.1.ToposText_Annotations_Book_4.csv")